In [24]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import numpy as np
import re
from shapely.geometry import Polygon, LineString, Point
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CocoDetection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.transforms as T
from torch.optim import SGD, Adam, Adadelta
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import transforms
from torch.utils.data._utils.collate import default_collate
import torchvision
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.transforms import functional as F
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import random
from math import radians, cos, sin
import cv2
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

### function to draw red bboxes around barlines

In [3]:
def draw_bounding_boxes(image_path):
    # Load the image in OpenCV
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or path is incorrect")

    # Convert the image from BGR to RGB (OpenCV uses BGR by default)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Define the color range for detection
    # Color #00ACC6 in RGB is (0, 172, 198)
    lower_bound = np.array([0, 172, 198])  # Lower bound of the color
    upper_bound = np.array([0, 172, 198])  # Upper bound of the color

    # Create a mask to detect specific color
    mask = cv2.inRange(image_rgb, lower_bound, upper_bound)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Convert the image to PIL format for easier drawing
    image_pil = Image.fromarray(image_rgb)
    draw = ImageDraw.Draw(image_pil)

    # Loop over the contours and draw bounding boxes
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        draw.rectangle([x, y, x + w, y + h], outline="red", width=2)

    # Display the image with bounding boxes
    image_pil.show()

    return image_pil

# Usage example
image_path = './data/ds2_dense/segmentation/lg-101766503886095953-aug-beethoven--page-4_seg.png'
result_image = draw_bounding_boxes(image_path)

### function to get barline bboxes for all images in a directory:

In [25]:
def analyze_image_for_barlines(image_path):
    """ Analyze an image to find barlines, return a DataFrame with bounding box details. """

    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or path is incorrect")
    
    height, width = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    lower_bound = np.array([0, 172, 198])  # Lower bound of the color
    upper_bound = np.array([0, 172, 198])  # Upper bound of the color

    mask = cv2.inRange(image_rgb, lower_bound, upper_bound)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    data = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w < 2 or h < 2:
            x = x - 1 if w < 2 else x
            y = y - 1 if h < 2 else y
            w = max(2, w)
            h = max(2, h)
        orthogonal_bbox = [float(x), float(y), float(x + w), float(y + h)]

        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int0(box)

        # Add each box to the data list
        data.append({
            'filename': image_path.split('/')[-1].replace('_seg',''),
            'a_bbox': orthogonal_bbox,
            'o_bbox': [float(coord) for xs in box.tolist() for coord in xs],
            'area': w*h,
            'width': width,
            'height': height
        })

    # Create a DataFrame from the collected data
    return pd.DataFrame(data)

### code to process an entire directory of images

In [26]:
# directory = './data/ds2_dense/segmentation/' 
# template = pd.DataFrame(columns=['filename','a_bbox','o_bbox','area','width','height','ann_id','label',
#                         'padded_bbox','duration','rel_position','duration_mask','rel_position_mask'])
# template.to_csv('./data/ds2_dense/barlines.csv')
# ann_id = -1
# for filename in tqdm(os.listdir(directory)):
#     if filename.endswith((".png", ".jpg", ".jpeg")):  # Check for image file extensions
#         file_path = os.path.join(directory, filename)
#         annotations = analyze_image_for_barlines(file_path)
#         if not annotations.empty:
#             annotations['ann_id'] = [ann_id - i for i in range(len(annotations))]
#             ann_id -= len(annotations)  # Decrement ann_id for next file
#             annotations['label'] = 156
#             annotations['padded_bbox'] = annotations['a_bbox']
#             annotations['duration'] = -1
#             annotations['rel_position'] = 0
#             annotations['duration_mask'] = 0
#             annotations['rel_position_mask'] = 0
#             annotations.to_csv('./data/ds2_dense/barlines.csv', mode='a', header=False)

In [27]:
def process_dataset(json_directory, segmentation_directory, output_directory):
    json_files = [f for f in os.listdir(json_directory) if f.endswith('.json')]
    
    for json_file in json_files:
        with open(os.path.join(json_directory, json_file), 'r') as file:
            data = json.load(file)
            images_df = pd.DataFrame(data['images'])
            filenames = images_df['filename'].tolist()

        output_csv = json_file.replace('.json', '.csv')
        output_path = os.path.join(output_directory, output_csv)

        # Prepare an empty DataFrame to collect all barline data
        barlines_data = pd.DataFrame(columns=['filename', 'a_bbox', 'o_bbox', 'area', 'width', 'height', 'ann_id',
                                              'label', 'padded_bbox', 'duration', 'rel_position', 'duration_mask', 
                                              'rel_position_mask'])
        barlines_data.to_csv(output_path)
        
        ann_id = -1
        # Process each image
        for filename in tqdm(filenames):
            file_path = os.path.join(segmentation_directory, filename.replace('.png','_seg.png'))
            annotations = analyze_image_for_barlines(file_path)
            
            # take annotation df and merge boxes with identical x coordinates
            
            if not annotations.empty:
                annotations['ann_id'] = [ann_id - i for i in range(len(annotations))]
                ann_id -= len(annotations)  # Decrement ann_id for next file
                annotations['label'] = 156
                annotations['padded_bbox'] = annotations['a_bbox']
                annotations['duration'] = -1
                annotations['rel_position'] = 0
                annotations['duration_mask'] = 0
                annotations['rel_position_mask'] = 0
                barlines_data = pd.concat([barlines_data, annotations], ignore_index=True)
        
        # Save all collected barline data to CSV after processing all files
        if not barlines_data.empty:
            barlines_data.to_csv(output_path, index=False)
            
# Example usage
json_directory = './data/ds2_dense/'
segmentation_directory = './data/ds2_dense/segmentation/'
output_directory = './data/ds2_dense/'
process_dataset(json_directory, segmentation_directory, output_directory)

100%|██████████████████████████████████████████| 352/352 [00:13<00:00, 26.82it/s]
